In [ ]:
# @title Install libraries

!pip install sentence_transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Load data

from io import BytesIO

import requests
import pandas as pd

gg_id = '814552626'
file_id = '11Xrvp1vdB0UJ34CIM_A-hKAnCJp_0BQA0HF6GGTerRc'
file_path = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gg_id}'

df = pd.read_csv(file_path)
df = df.ffill()
df['Id'] = df.reset_index(drop=False)['index'].astype(str).str.zfill(5)
df

,Object,Detail,Interpretation,Id
0,Ảnh,Đốt ảnh,Tình duyên sắp thành,00000
1,Ảnh,Tặng ảnh,Điềm chia ly / sắp đi xa,00001
2,Ảnh,Được tặng ảnh,Điềm chia ly / sắp đi xa,00002
3,Ảnh,Ảnh người yêu,Có tình địch,00003
4,Ảnh,Xé ảnh người khác,Sắp hội ngộ người đó,00004
...,...,...,...,...
2303,Xưởng,Trông nom xưởng thở,Tài lộc hanh thông,02303
2304,Xưởng,Đi ra khỏi xưởng,Bạn sẽ được quý nhân cất nhắc,02304
2305,Xưởng,Đi ra khỏi xưởng rồi quay lại,Bạn đang được chủ tin cậy,02305
2306,Yêu,Yêu tha thiết 1 người mà người ấy luôn ruồng rẫy,Hôn nhân sắp thành,02306


In [ ]:
doc_ids = df['Id'].tolist()
documents = [d.lower() for d in df['Detail'].tolist()]
metadatas = df.drop(columns=['Id','Detail']).to_dict(orient='records')
metadatas[:5]

[{'Object': 'Ảnh', 'Interpretation': 'Tình duyên sắp thành'},
 {'Object': 'Ảnh', 'Interpretation': 'Điềm chia ly / sắp đi xa'},
 {'Object': 'Ảnh', 'Interpretation': 'Điềm chia ly / sắp đi xa'},
 {'Object': 'Ảnh', 'Interpretation': 'Có tình địch'},
 {'Object': 'Ảnh', 'Interpretation': 'Sắp hội ngộ người đó'}]

In [ ]:
# @title Load embedding model
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction as EmbeddingFunction

model_name = 'bkai-foundation-models/vietnamese-bi-encoder'
embed_func = EmbeddingFunction(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
# @title Build database
# Document: https://docs.trychroma.com/getting-started
%%time

import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection(name="dreamerpretation", embedding_function=embed_func)

collection.add(
    documents=documents,
    ids=doc_ids,
    metadatas=metadatas
)

collection.__dict__

{'_client': <chromadb.api.segment.SegmentAPI at 0x7f71084dfca0>,
 '_model': Collection(id=UUID('def8c49e-554e-4b02-be94-83ba34f7a997'), name='dreamerpretation', configuration_json={'hnsw_configuration': {'space': 'l2', 'ef_construction': 100, 'ef_search': 10, 'num_threads': 2, 'M': 16, 'resize_factor': 1.2, 'batch_size': 1000, 'sync_threshold': 100, '_type': 'HNSWConfigurationInternal'}, '_type': 'CollectionConfigurationInternal'}, metadata=None, dimension=768, tenant='default_tenant', database='default_database', version=0),
 '_embedding_function': <chromadb.utils.embedding_functions.sentence_transformer_embedding_function.SentenceTransformerEmbeddingFunction at 0x7f724a0c1c30>,
 '_data_loader': None}

In [ ]:
# @title Inference
%%time

queries = ["con chó"]
Q = len(queries)
N = 10
results = collection.query(query_texts=queries, n_results=N)

q_idx = 0
results_processed = dict()
for k, v in results.items():
    if v is None:
        continue
    if k == 'metadatas':
        for d_ in v[q_idx]:
            for k_, v_ in d_.items():
                if k_ not in results_processed.keys():
                    results_processed[k_] = [v_]
                else:
                    results_processed[k_] += [v_]
    else:
        results_processed[k] = v[q_idx]

results_df = pd.DataFrame.from_records(results_processed)
results_df

CPU times: user 154 ms, sys: 5.45 ms, total: 159 ms
Wall time: 385 ms


,Interpretation,Object,distances,documents,ids,included
0,Được quý nhân nâng đỡ,Chó,8.617681,chó,00451,metadatas
1,Điềm báo xấu về 1 mối quan hệ,Chó,26.473103,chó dữ,00446,metadatas
2,Điềm báo tốt lành về 1 mối quan hệ giao hảo hữ...,Chó,27.400896,chó hiền,00445,metadatas
3,Sắp được thỏa mãn chuyện tình cảm,Chó,29.130863,chó cái,00453,metadatas
4,"Sắp có tranh chấp, bất hòa",Chó,33.308838,nuôi chó,00450,metadatas
5,Cẩn thận những người bạn tin tưởng sẽ bội bạc ...,Mèo,35.255753,con mèo,01355,metadatas
6,Sắp nhận được 1 món tiền nhỏ,Chó,35.388763,chó vàng,00447,metadatas
7,Có sự do dự trong công việc,Cọp,35.702271,con cọp,00506,metadatas
8,Sắp nhận được 1 món tiền nhỏ,Chó,37.808250,chó nâu,00448,metadatas
9,Cẩn thận đề phòng mất tiền của,Chó,38.633263,bị chó sủa,00455,metadatas
